Arthur Rezende Salles da Costa - 166003 

T2 - LA2ML


In [47]:
import pandas as pd 
import numpy as np
import os 


Implementacao SVD

In [154]:
A = np.array([[1, 0, 1], [-2, 1, 0]])


def my_svd(A):

    ata = np.dot(A.T, A)

    eigenvalues, eigenvectors = np.linalg.eigh(ata)
    sing_values = np.nan_to_num(np.sqrt(eigenvalues), True, 0)
    sorted_singular_values = np.argsort(sing_values)[::-1]

    sigma = np.eye(A.shape[0], A.shape[1]) * sing_values[sorted_singular_values]
    sigma = np.nan_to_num(sigma, False, 0)
    # print(sigma)

    V = eigenvectors[:, sorted_singular_values]
    U = np.dot(A, V) / sing_values[sorted_singular_values]

    if A.shape[0] != A.shape[1]:
        U = U[:, :-1]

    return U, sigma, V.T

u, sigma, vt = my_svd(A)
u, sigma, vt

C:\Users\Arthur\AppData\Local\Temp\ipykernel_23760\1596722193.py:9: RuntimeWarning: invalid value encountered in sqrt
  sing_values = np.nan_to_num(np.sqrt(eigenvalues), True, 0)
C:\Users\Arthur\AppData\Local\Temp\ipykernel_23760\1596722193.py:17: RuntimeWarning: divide by zero encountered in divide
  U = np.dot(A, V) / sing_values[sorted_singular_values]


(array([[-0.4472136 ,  0.89442719],
        [ 0.89442719,  0.4472136 ]]),
 array([[2.44948974, 0.        , 0.        ],
        [0.        , 1.        , 0.        ]]),
 array([[-9.12870929e-01,  3.65148372e-01, -1.82574186e-01],
        [-5.78877993e-17,  4.47213595e-01,  8.94427191e-01],
        [ 4.08248290e-01,  8.16496581e-01, -4.08248290e-01]]))

In [155]:

u, sigma, vt = np.linalg.svd(A)
u, sigma, vt

(array([[-0.4472136 ,  0.89442719],
        [ 0.89442719,  0.4472136 ]]),
 array([2.44948974, 1.        ]),
 array([[-9.12870929e-01,  3.65148372e-01, -1.82574186e-01],
        [-3.73536832e-16,  4.47213595e-01,  8.94427191e-01],
        [-4.08248290e-01, -8.16496581e-01,  4.08248290e-01]]))

Leitura dos Dados

In [16]:
def read_data_ml100k():
    # code extracted from -> https://www.d2l.ai/chapter_recommender-systems/movielens.html
    data_dir = "../t2/data/ml-100k"
    names = ['user_id', 'item_id', 'rating', 'timestamp']
    data = pd.read_csv(os.path.join(data_dir, 'u.data'), sep='\t',
                       names=names, engine='python')
    num_users = data.user_id.unique().shape[0]
    num_items = data.item_id.unique().shape[0]
    return data, num_users, num_items

data, num_users, num_items = read_data_ml100k()
print(num_users, num_items)
display(data.head())

943 1682


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
